In [ ]:
%matplotlib inline


# Simulate dipole


This example demonstrates how to simulate a dipole using the Neuron
simulator.



In [ ]:
# Authors: Mainak Jas <mainak.jas@telecom-paristech.fr>
#          Sam Neymotin <samnemo@gmail.com>

import os.path as op

Let us import mne_neuron



In [ ]:
import mne_neuron
from mne_neuron.paramrw import ExpParams
from mne_neuron import simulate_dipole

from neuron import h

Then we setup the directories and Neuron



In [ ]:
mne_neuron_root = op.join(op.dirname(mne_neuron.__file__), '..')
h.load_file("stdrun.hoc")
h.nrn_load_dll(op.join(mne_neuron_root, "x86_64/.libs/libnrnmech.so"))

Then we setup the directories



In [ ]:
params_fname = op.join(mne_neuron_root, 'param', 'default.param')
p_exp = ExpParams(params_fname)
params = p_exp.return_pdict('default', 0)

Now let's simulate the dipole and plot it



In [ ]:
dpl = simulate_dipole(params)
dpl.plot()